문제 (week5 input.txt)  
출현하는 단어의 첫 번째 두 글자의 출현빈도를 세고 출현 빈도가 32보다 작은 (미만; <) 단어들의 출현빈도의 합은 이다  

대소문자 구별하지 않음    
첫 두 글자를 카운트하므로 길이가 2와 같거나 보다 긴 단어만 대상으로 함    
공백은 당연 제외    
문장부호(, . ? ‘ “ 등)은 따로 처리하지 않음   

===> slicing을 통해 단어의 2개 글자만 뽑아내고, reduceByKey()하면 되는데 삽질....  
===> 대소문자 구별하지 않음 의미파악 삽질.....

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "transformation 1")

In [3]:
# flatMap, filter이용 공백단어는 제거 
readfile=sc.textFile("input.txt").flatMap(lambda x: x.split(" ")).filter(lambda x: x !="")
#readfile.collect()
# ['A',
#  'STUDY',
#  'IN',
#  'SCARLET.',
#  'PART',
#  'I.',
#  '(Being',
#  'a',,,,]

#  첫글자에 공백있는 단어 있을 수 있는가?
#  '“Looking',
#  'for',
#  'lodgings.”',


In [4]:
# 단어중 공백 있을 수 있어 제외  # 1문자인 것은 제외, 두 글자이상만 
datas=readfile.map(lambda x: x.strip()).filter(lambda x: len(x)>=2)
#datas.collect()
# ['STUDY',
#  'IN',
#  'SCARLET.',
#  'PART',
#  'I.',
#  '(Being',
#  'reprint',
#  'from',,,]

In [5]:
# 첫번째 두글자만 추출(slicing) 한 list 생성
keys = datas.map(lambda x : x[:2])
# ['ST',
#  'IN',
#  'SC',
#  'PA',
#  'I.',
#  '(B',
#  're',
#  'fr',
#  'th',,,,]
# keys.collect()


In [8]:
# 대소문자구별하지 않으므로 소문자로 변경
keys_lower = keys.map(lambda x: x.lower())
# keys_lower.collect()
# ['st',
#  'in',
#  'sc',
#  'pa',
#  'i.',
#  '(b',
#  're',
#  'fr',
#  'th',,,]


In [13]:
keys_tuple = keys_lower.map(lambda x:(x, 1))
# keys_tuple.collect()
# [('st', 1),
#  ('in', 1),
#  ('sc', 1),
#  ('pa', 1),
#  ('i.', 1),
#  ('(b', 1),
#  ('re', 1),
#  ('fr', 1),,,]

In [15]:
keys_tuple_reducebykey= keys_tuple.reduceByKey(lambda a,b : a+b)
# keys_tuple_reducebykey.collect()
# [('st', 515),
#  ('in', 1076),
#  ('sc', 70),
#  ('pa', 188),
#  ('i.', 9),
#  ('(b', 1),
#  ('re', 456),
#  ('fr', 270),
#  ('th', 4546),,,]

In [18]:
data_filtered_32 = keys_tuple_reducebykey.filter(lambda v: v[1] < 32)
# data_filtered_32.collect()
# [('i.', 9),
#  ('(b', 1),
#  ('h.', 2),
#  ('m.', 2),
#  ('18', 2),
#  ('gh', 7),
#  ('“o', 19),
#  ('ir', 14),
#  ('ki', 19),;;]

In [19]:
type(data_filtered_32.values().collect()) # list
sum (data_filtered_32.values().collect())

830

## python datatype으로 변경하여 검증

In [22]:
keys_list = keys.map(lambda x:x).collect()
keys_list = [ key.lower() for key in keys_list ]
print(type(keys_list)) # list

import pandas as pd
import numpy as np
ser = pd.Series(keys_list)
print(ser.head())
    # 0    ST
    # 1    IN
    # 2    SC
    # 3    PA
    # 4    I.
    # dtype: object
type(ser.value_counts().values) # <class 'list'>
print(ser.value_counts().values[:10])
    # [4167 1672 1248 1245 1105 1027 1012  968  921  809  745  624  540  527
    #   507  487  477  472  455  441  440  422  408  394  385  371  366  363
    #   361  338  333  321  319  313  309  293  293  2 ,,,]
sum(list( filter(lambda x:x<32, ser.value_counts().values ) ))
# 830

<class 'list'>
0    st
1    in
2    sc
3    pa
4    i.
dtype: object
[4546 1689 1258 1256 1205 1135 1076 1006  937  888]


830

In [27]:
datas_list=datas.map(lambda x:x).collect()
datas_list= [ key.lower() for key in datas_list ]
print(type(datas_list)) # list

keys_list = keys.map(lambda x:x).collect()
keys_list = [ key.lower() for key in keys_list ]
print(type(keys_list)) # list

# count위한 dict 생성
kv_dict1 = dict(map(lambda x: (x,0), keys_list))

for key, _ in kv_dict1.items():
    for i in datas_list:
#         print(f'{key}:{i}')
#         tmp_list=[]
        if key == i[0:2] : # str compare
#             tmp_list.append(i)
            kv_dict1[key] +=1 
#         print(tmp_list)
filtered_list1 = list( filter(lambda x: x <32, kv_dict1.values()) )
print(sum(filtered_list1)) # 830

<class 'list'>
<class 'list'>
830


In [29]:
datas_list=datas.map(lambda x:x).collect()
datas_list= [ key.lower() for key in datas_list ]
print(type(datas_list)) # list

keys_list = keys.map(lambda x:x).collect()
keys_list = [ key.lower() for key in keys_list ]
print(type(keys_list)) # list

# count위한 dict 생성
kv_dict2 = dict(map(lambda x: (x,0), keys_list))

# 알고리즘 
for key, _ in kv_dict2.items():
    for i in datas_list:
#         print(f'{key}:{i}')
#         tmp_list=[]
        if key == i[0:2] : # str compare
#             tmp_list.append(i)
            kv_dict2[key] +=1 
#         print(tmp_list)
filtered_list2 = list( filter(lambda x: x <32, kv_dict2.values()) )
sum(filtered_list2) # 830


<class 'list'>
<class 'list'>


830

In [ ]:
# data_tuple.foreach(lambda x: x[1] += 1 if x[0] in datas) # error
# data_tuple.foreach(lambda x: x[1].mapValues(lambda v: v+1) if x[0] in datas )  # error
# data_tuple.mapValues(lambda x: x[1]+=1 if x[0] in datas )  # error
# data_tuple.foreach(lambda x: x[1].mapValues(lambda v: v+1) if x[0] in datas )  # error
# datas.eachfor(lambda x: x.contains(data_list.map(lambda v:v)))  # error
# data_=data_tuple.pipe("lambda x: x[1].mapValues(lambda v: v+1) if x[0] in datas")
# datas = datas.map(lambda x:(x,1))
# datas.cogroup(data_tuple)

In [ ]:
# data_tuple.filter(lambda x: x[0].contains(datas.map(lambda v:v)))
# data_=data_tuple.filter(lambda x: x[0].contains("S"))
# data_=data_.mapValues(lambda x:x[1]+1)
# data_.collect()

In [ ]:
# key, value에서 value에 의한 정렬
# sortbyValue() 없음 # sortByKey()  #sortBy 있음
data_sortby=data_reducebykey.sortBy(lambda a: a[1])
#data_sortby.collect()
# [('ST', 1),
#  ('(B', 1),
#  ('Gh', 1),
#  ('Em', 1),
#  ('“p', 1),
#  ('Vo', 1),
#  ('Bi', 1),,,,]

In [ ]:
# mapValue()
# data_mapvalue = data_reducebykey.mapValues( lambda values: map(lambda v:filter(v)<=32 , values )) # Py4JJavaError
# data_mapvalue = data_reducebykey.mapValues( lambda v: v+5 ) # ok

data_mapvalue = data_reducebykey.mapValues(lambda v: v<=32 )  
# [('ST', True),
#  ('IN', True),
#  ('SC', True),
#  ('PA', True),
#  ('I.', True),
#  ('(B', True),
#  ('re', False),
#  ('fr', False),
print(type(data_mapvlaue)) # pyspark.rdd.PipelinedRDD
# data_mapvalue.collect()
#data_mapvalue.collect()


In [ ]:
sc.stop()